In [1]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import argparse
import torch.nn.functional as F
from torch.utils.data import DataLoader
from functools import partial

import net
from hyptorch.pmath import dist_matrix
from proxy_anchor import dataset
from proxy_anchor.utils import calc_recall_at_k
from sampler import UniqueClassSempler
from proxy_anchor.dataset import CUBirds, SOP, Cars
from proxy_anchor.dataset.Inshop import Inshop_Dataset

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '6,7'
seed = 1
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [3]:
path = '/data/xuyunhao/datasets'
ds = 'CUB'
num_samples = 2
bs = 200
lr = 1e-5
t = 0.2
emb = 384
ep = 100
local_rank = 0
workers = 8
optimizer = 'adamw'
lr_decay_step = 10
lr_decay_gamma = 0.5

model =  'resnet50'
hyp_c = 0.1
clip_r  = 2.3
resize = 224
crop = 224
gpu_id = -1
bn_freeze = 1

# 投影超球模型

In [4]:
def project(x, *, c=1.0):
    c = torch.as_tensor(c).type_as(x)
    return _project(x, c)

def _project(x, c):
    norm = torch.clamp_min(x.norm(dim=-1, keepdim=True, p=2), 1e-5)
    maxnorm = (1 - 1e-3) / (c ** 0.5)
    cond = norm > maxnorm
    projected = x / norm * maxnorm
    return torch.where(cond, projected, x)

def dexp0(u, *, c=1.0):
    c = torch.as_tensor(c).type_as(u)
    return _dexp0(u, c)

def _dexp0(u, c):
    sqrt_c = c ** 0.5
    u_norm = torch.clamp_min(u.norm(dim=-1, p=2, keepdim=True), 1e-5)
    gamma_1 = torch.tan(sqrt_c * u_norm) * u / (sqrt_c * u_norm)
    return gamma_1

def _dist_matrix_d(x, y, c):
    xy =torch.einsum("ij,kj->ik", (x, y))  # B x C
    x2 = x.pow(2).sum(-1, keepdim=True)  # B x 1
    y2 = y.pow(2).sum(-1, keepdim=True)  # C x 1
    sqrt_c = c ** 0.5
    num1 = 2*c*(x2+y2.permute(1, 0)-2*xy)+1e-5
    num2 = torch.mul((1+c*x2),(1+c*y2.permute(1, 0)))
    return (1/sqrt_c * torch.acos(1-num1/num2))


def dist_matrix_d(x, y, c=1.0):
    c = torch.as_tensor(c).type_as(x)
    return _dist_matrix_d(x, y, c)

class ToProjection_hypersphere(nn.Module):
    def __init__(self, c, clip_r=None):
        super(ToProjection_hypersphere, self).__init__()
        self.register_parameter("xp", None)
        self.c = c
        self.clip_r = clip_r
        
    def forward(self, x):
        if self.clip_r is not None:
            x_norm = torch.norm(x, dim=-1, keepdim=True) + 1e-5
            fac =  torch.minimum(
                torch.ones_like(x_norm), 
                self.clip_r / x_norm
            )
            x = x * fac
        return project(dexp0(x, c=self.c), c=self.c)

# 损失函数

In [5]:
def contrastive_loss(x0, x1, tau, hyp_c):
    # x0 and x1 - positive pair
    # tau - temperature
    # hyp_c - hyperbolic curvature, "0" enables sphere mode

    dist_f = lambda x, y: -dist_matrix_d(x, y, c=hyp_c)
    bsize = x0.shape[0]
    target = torch.arange(bsize).cuda()
    eye_mask = torch.eye(bsize).cuda() * 1e9
    logits00 = dist_f(x0, x0) / tau - eye_mask
    logits01 = dist_f(x0, x1) / tau
    logits = torch.cat([logits01, logits00], dim=1)
    logits -= logits.max(1, keepdim=True)[0].detach()
    loss = F.cross_entropy(logits, target)
    stats = {
        "logits/min": logits01.min().item(),
        "logits/mean": logits01.mean().item(),
        "logits/max": logits01.max().item(),
        "logits/acc": (logits01.argmax(-1) == target).float().mean().item(),
    }
    return loss, stats

# 模型

In [6]:
import torch
import torch.nn as nn
import math
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import torch.nn.init as init
import hyptorch.nn as hypnn
from torchvision.models import resnet18
from torchvision.models import resnet34
from torchvision.models import resnet50
from torchvision.models import resnet101

## Resnet50

In [7]:
class Resnet50(nn.Module):
    def __init__(self,embedding_size, pretrained=True, bn_freeze = True, hyp_c = 0, clip_r = 0):
        super(Resnet50, self).__init__()

        self.model = resnet50(pretrained)
        self.embedding_size = embedding_size
        self.hyp_c = hyp_c
        self.clip_r = clip_r
        self.num_ftrs = self.model.fc.in_features
        self.model.gap = nn.AdaptiveAvgPool2d(1)
        self.model.gmp = nn.AdaptiveMaxPool2d(1)

        self.Dlayer = ToProjection_hypersphere(
            c=self.hyp_c,
            clip_r=self.clip_r,
        )
        self.model.embedding = nn.Sequential(nn.Linear(self.num_ftrs, self.embedding_size), self.Dlayer)

        self._initialize_weights()

        if bn_freeze:
            for m in self.model.modules():
                if isinstance(m, nn.BatchNorm2d):
                    m.eval()
                    m.weight.requires_grad_(False)
                    m.bias.requires_grad_(False)

    def forward(self, x):
        x = self.model.conv1(x)
        x = self.model.bn1(x)
        x = self.model.relu(x)
        x = self.model.maxpool(x)
        x = self.model.layer1(x)
        x = self.model.layer2(x)
        x = self.model.layer3(x)
        x = self.model.layer4(x)

        avg_x = self.model.gap(x)
        max_x = self.model.gmp(x)

        x = max_x + avg_x
        x = x.view(x.size(0), -1)
        x_d = self.model.embedding(x)
        return x_d

    def _initialize_weights(self):
        init.kaiming_normal_(self.model.embedding[0].weight, mode='fan_out')
        init.constant_(self.model.embedding[0].bias, 0)

## Resnet101

In [8]:
class Resnet101(nn.Module):
    def __init__(self,embedding_size, pretrained=True, bn_freeze = True, hyp_c = 0, clip_r = 0):
        super(Resnet101, self).__init__()

        self.model = resnet101(pretrained)
        self.embedding_size = embedding_size
        self.hyp_c = hyp_c
        self.clip_r = clip_r
        self.num_ftrs = self.model.fc.in_features
        self.model.gap = nn.AdaptiveAvgPool2d(1)
        self.model.gmp = nn.AdaptiveMaxPool2d(1)

        self.Dlayer = ToProjection_hypersphere(
            c=self.hyp_c,
            clip_r=self.clip_r,
        )
        self.model.embedding = nn.Sequential(nn.Linear(self.num_ftrs, self.embedding_size), self.Dlayer)

        self._initialize_weights()

        if bn_freeze:
            for m in self.model.modules():
                if isinstance(m, nn.BatchNorm2d):
                    m.eval()
                    m.weight.requires_grad_(False)
                    m.bias.requires_grad_(False)

    def forward(self, x):
        x = self.model.conv1(x)
        x = self.model.bn1(x)
        x = self.model.relu(x)
        x = self.model.maxpool(x)
        x = self.model.layer1(x)
        x = self.model.layer2(x)
        x = self.model.layer3(x)
        x = self.model.layer4(x)

        avg_x = self.model.gap(x)
        max_x = self.model.gmp(x)

        x = max_x + avg_x
        x = x.view(x.size(0), -1)
        x_d = self.model.embedding(x)
        return x_d

    def _initialize_weights(self):
        init.kaiming_normal_(self.model.embedding[0].weight, mode='fan_out')
        init.constant_(self.model.embedding[0].bias, 0)

In [9]:
def evaluate(get_emb_f, ds_name, hyp_c):
    emb_head = get_emb_f(ds_type="eval")
    recall_head = get_recall(*emb_head, ds_name, hyp_c)
    return recall_head

def get_recall(p, y, ds_name, hyp_c):
    if ds_name == "CUB" or ds_name == "Cars":
        k_list = [1, 2, 4, 8, 16, 32]
    elif ds_name == "SOP":
        k_list = [1, 10, 100, 1000]

    dist_m = torch.empty(len(p), len(p), device="cuda")
    for i in range(len(p)):
        dist_m[i : i + 1] = -dist_matrix(p[i : i + 1], p, hyp_c)

    y_cur = y[dist_m.topk(1 + max(k_list), largest=True)[1][:, 1:]]
    y = y.cpu()
    y_cur = y_cur.float().cpu()
    recall = [calc_recall_at_k(y, y_cur, k) for k in k_list]
    print(recall)
    return recall[0]

def get_emb(
    model,
    ds,
    path,
    ds_type="eval",
    world_size=1,
    num_workers=8,
):
    eval_tr = dataset.utils.make_transform(
        is_train = True, 
        is_inception = (model == 'bn_inception')
    )
    ds_eval = ds(path, ds_type, eval_tr)
    if world_size == 1:
        sampler = None
    else:
        sampler = torch.utils.data.distributed.DistributedSampler(ds_eval)
    dl_eval = DataLoader(
        dataset=ds_eval,
        batch_size=100,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
        drop_last=False,
        sampler=sampler,
    )
    model.eval()
    p, y = eval_dataset(model, dl_eval)
    y = y.cuda()
    model.train()
    return p, y

def eval_dataset(model, dl):
    all_xp, all_y = [], []
    for x, y in dl:
        with torch.no_grad():
            x = x.cuda(non_blocking=True)
            p= model(x)
            all_xp.append(p)
        all_y.append(y)
    return torch.cat(all_xp), torch.cat(all_y)

In [10]:
world_size = int(os.environ.get("WORLD_SIZE", 1))

train_tr = dataset.utils.make_transform(
    is_train = True, 
    is_inception = (model == 'bn_inception')
)

ds_list = {"CUB": CUBirds, "SOP": SOP, "Cars": Cars, "Inshop": Inshop_Dataset}
ds_class = ds_list[ds]
ds_train = ds_class(path, "train", train_tr)

sampler = UniqueClassSempler(
    ds_train.ys, num_samples, local_rank, world_size
)
dl_train = DataLoader(
    dataset=ds_train,
    sampler=sampler,
    batch_size=bs,
    num_workers=workers,
    pin_memory=True,
)

if model.find('resnet18')+1:
    model = Resnet18(embedding_size=emb, pretrained=True, bn_freeze = bn_freeze,hyp_c = hyp_c, clip_r = clip_r).cuda().train() 
elif model.find('resnet34')+1:
    model = Resnet34(embedding_size=emb, pretrained=True, bn_freeze = bn_freeze,hyp_c = hyp_c, clip_r = clip_r).cuda().train() 
elif model.find('resnet50')+1:
    model = Resnet50(embedding_size=emb, pretrained=True, bn_freeze = bn_freeze,hyp_c = hyp_c, clip_r = clip_r) 
elif model.find('resnet101')+1:
    model = Resnet101(embedding_size=emb, pretrained=True, bn_freeze = bn_freeze,hyp_c = hyp_c, clip_r = clip_r)

if gpu_id == -1:
    model = nn.DataParallel(model)
    
model.cuda().train()

loss_f = partial(contrastive_loss, tau=t, hyp_c= hyp_c)

get_emb_f = partial(
    get_emb,
    model=model,
    ds=ds_class,
    path=path,
    num_workers=workers,
    world_size=world_size,
)
optimizer = optim.AdamW(model.module.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=lr_decay_step, gamma = lr_decay_gamma)
print("Training for {} epochs.".format(ep))

r0= evaluate(get_emb_f, ds, hyp_c=hyp_c)
print("The recall before train: ", r0)

losses_list = []
best_recall= 0
best_epoch = 0

for epoch in range(0, ep):
    model.train()
    if bn_freeze:
        modules = model.module.model.modules()
        for m in modules: 
            if isinstance(m, nn.BatchNorm2d):
                m.eval()

    losses_per_epoch = []
    sampler.set_epoch(epoch)
    stats_ep = []
    for x, y in dl_train:
        y = y.view(len(y) // num_samples, num_samples)
        assert (y[:, 0] == y[:, -1]).all()
        s1 = y[:, 0].tolist()
        assert len(set(s1)) == len(s1)

        x = x.cuda(non_blocking=True)
        d = model(x)
        d = d.view(len(x) // num_samples, num_samples, emb)
        loss = 0
        for i in range(num_samples):
            for j in range(num_samples):
                if i != j:
                    l, st = loss_f(d[:, i], d[:, j])
                    loss += l
                    stats_ep.append({**st, "loss": l.item()})

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 10)
        optimizer.step()
        
    scheduler.step()        
    rh= evaluate(get_emb_f, ds, hyp_c = hyp_c)
    stats_ep = {k: np.mean([x[k] for x in stats_ep]) for k in stats_ep[0]}
    stats_ep = {"recall": rh, **stats_ep}
    if rh > best_recall :
        best_recall = rh
        best_epoch = epoch
    print("epoch:",epoch,"recall: ", rh)
    print("best epoch:",best_epoch,"best recall: ", best_recall)

Training for 100 epochs.
[0.40698852126941254, 0.5379810938555031, 0.6617150573936529, 0.7670492910195814, 0.8583727211343687, 0.9225185685347738]
The recall before train:  0.40698852126941254
[0.4176232275489534, 0.5416948008102633, 0.6637407157326131, 0.7788656313301823, 0.8609047940580689, 0.9188048615800135]
epoch: 0 recall:  0.4176232275489534
best epoch: 0 best recall:  0.4176232275489534
[0.4348413234301148, 0.5589128966914247, 0.6806212018906145, 0.7839297771775827, 0.8652937204591492, 0.9228561782579339]
epoch: 1 recall:  0.4348413234301148
best epoch: 1 best recall:  0.4348413234301148
[0.43821742066171504, 0.5666779203241054, 0.6836596893990546, 0.7798784604996624, 0.8641120864280891, 0.9221809588116138]
epoch: 2 recall:  0.43821742066171504
best epoch: 2 best recall:  0.43821742066171504
[0.44480081026333557, 0.5712356515867657, 0.6887238352464551, 0.7879810938555031, 0.8666441593517893, 0.9225185685347738]
epoch: 3 recall:  0.44480081026333557
best epoch: 3 best recall:  0

[0.4637069547602971, 0.5921674544226874, 0.6949696151249156, 0.787137069547603, 0.861748818365969, 0.9145847400405132]
epoch: 39 recall:  0.4637069547602971
best epoch: 37 best recall:  0.47788656313301825
[0.474510465901418, 0.5941931127616475, 0.7003713706954761, 0.7959149223497637, 0.8646185010128292, 0.9188048615800135]
epoch: 40 recall:  0.474510465901418
best epoch: 37 best recall:  0.47788656313301825
[0.47889939230249834, 0.5935178933153274, 0.7002025658338961, 0.7933828494260635, 0.8664753544902093, 0.9181296421336934]
epoch: 41 recall:  0.47889939230249834
best epoch: 41 best recall:  0.47889939230249834
[0.462694125590817, 0.5806887238352465, 0.6961512491559757, 0.7945644834571236, 0.8649561107359892, 0.9134031060094531]
epoch: 42 recall:  0.462694125590817
best epoch: 41 best recall:  0.47889939230249834
[0.47822417285617824, 0.5987508440243079, 0.699696151249156, 0.7967589466576638, 0.8678257933828494, 0.9216745442268738]
epoch: 43 recall:  0.47822417285617824
best epoch: 

[0.4925725860904794, 0.6049966239027684, 0.7086428089128967, 0.800810263335584, 0.8698514517218096, 0.9179608372721134]
epoch: 79 recall:  0.4925725860904794
best epoch: 79 best recall:  0.4925725860904794
[0.45948683322079675, 0.5850776502363269, 0.6964888588791357, 0.7952397029034436, 0.8685010128291695, 0.9174544226873734]
epoch: 80 recall:  0.45948683322079675
best epoch: 79 best recall:  0.4925725860904794
[0.4601620526671168, 0.5828831870357867, 0.699020931802836, 0.7928764348413234, 0.8669817690749494, 0.9134031060094531]
epoch: 81 recall:  0.4601620526671168
best epoch: 79 best recall:  0.4925725860904794
[0.4775489534098582, 0.5980756245779878, 0.7027346387575962, 0.7942268737339635, 0.861917623227549, 0.9150911546252533]
epoch: 82 recall:  0.4775489534098582
best epoch: 79 best recall:  0.4925725860904794
[0.4724848075624578, 0.5960499662390277, 0.7039162727886563, 0.7922012153950033, 0.863436866981769, 0.9115462525320729]
epoch: 83 recall:  0.4724848075624578
best epoch: 79 